In [26]:
import pandas as pd
import requests
import json
from requests.auth import HTTPBasicAuth
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException

In [23]:
API_KEY = "ZcZlIC8XY0iSqwFaoQHei9L20VhraaIaW5WJIMPOlTo7"
SERVICE_URL = "https://api.eu-de.assistant.watson.cloud.ibm.com"
API_APP_URL = "https://api.eu-de.assistant.watson.cloud.ibm.com/instances/6833047d-680e-4bd1-ace3-2a9e87387e14"
API_ASSISTANT_ID = "bfe2fe18-c68e-4f54-9891-7bbcc978c76a"
API_VERSION = "2021-11-27"

In [5]:
test    = pd.read_csv('./testset.csv')

In [8]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = 'â–', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [78]:
authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV2(
    version=API_VERSION,
    authenticator = authenticator
)

assistant.set_service_url(SERVICE_URL)
try:
    response = assistant.create_session(
        assistant_id=API_ASSISTANT_ID
    ).get_result()
except ApiException as ex:
    print("Method failed with status code " + str(ex.code) + ": " + ex.message)

session_id = json.dumps(response['session_id'])
print(session_id)

"8d5af7fa-6a54-4c37-be6e-2da264ab814c"


In [79]:
example_count = test.shape[0]
correctly_detected = 0
incorrects = []

for i in range(example_count):
    query = test['Query'][i]
    ground_true = test['Category'][i].replace(' ', '_')
    print(query, ground_true)
    try:
        response = assistant.message(
            assistant_id=API_ASSISTANT_ID,
            session_id=session_id.strip('"'),
            input={
                'message_type': 'text',
                'text': query
            }
        ).get_result()

        answer = json.dumps(response['output']['generic'][0]['text'])
        if len(response["output"]["intents"]) > 0:
            intent = json.dumps(response['output']['intents'][0]['intent'])
            confidence = json.dumps(response['output']['intents'][0]['confidence'])
            print(intent, confidence)    
            if(str(intent).strip('"') == ground_true):
                correctly_detected = correctly_detected + 1
            else:
                incorrects.append(str(query + ',' + intent + ',' + ground_true))
        else:
            incorrects.append(str(query + ',[]' + ',' + ground_true))
    except ApiException as ex:
        print("Method failed with status code " + str(ex.code) + ": " + ex.message)



What ID do I need to show at the test centre? Test_center
"Test_center" 0.962119483947754
Can I bring some supplies with me? Test_center
Can the calculator be used in the test centre? Test_center
"Test_center" 0.9506243228912354
What tools can I bring with me to the test centre? Test_center
"Test_center" 0.943186092376709
How does it work in the test centre? Test_center
"Test_center" 0.9805212497711182
What does the test centre fall under? Test_center
"Test_center" 0.9770933628082277
Who is in charge of the test centre? Test_center
"Test_center" 0.9610862731933594
Which faculty can use the testing center? Test_center
"Test_center" 0.8064403533935547
I missed a test date, I failed my test? Test_center
"Test_center" 0.9576482772827148
Are there any resources or support available for students who have been denied admission? admission
"admission" 0.892021656036377
Are there any opportunities for students to appeal the admissions decision? admission
"admission" 0.883009672164917
What are st

In [81]:
print('Number of test examples: ', example_count)
print('Correctly detected: ', correctly_detected)
print('Incorrectly detected: ', example_count - correctly_detected)
print('Accuracy: ', correctly_detected / example_count)
print('Incorrectly_detected: ', incorrects)

Number of test examples:  27
Correctly detected:  22
Incorrectly detected:  5
Accuracy:  0.8148148148148148
Incorrectly_detected:  ['Can I bring some supplies with me?,[],Test_center', 'What are study programs available?,"study_management",admission', 'Whom should I contact in case of emergency in dormitories,"emergency",accomodation', 'Are there any opportunities for students to participate in international sports competitions or events?,"activities_and_events",sports', 'Where can I get wheelchain inside the university?,"activities_and_events",emergency']
